In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

data = pd.read_csv('/teamspace/studios/this_studio/data_new/metadata.csv')
# print(data.columns)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396000 entries, 0 to 395999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   method      396000 non-null  object
 1   SNR         396000 non-null  int64 
 2   target      396000 non-null  object
 3   image_path  396000 non-null  object
dtypes: int64(1), object(3)
memory usage: 12.1+ MB


In [3]:
from datasets import *

# 创建数据集
dataset = PairedDataset(
    csv_file='/teamspace/studios/this_studio/data_new/metadata.csv',
    methods=['CWT', 'STFT'],
    snr='20',
    transform=transform,
)

# 数据集划分：80%训练集，20%验证集
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=os.cpu_count())
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=os.cpu_count())

# 使用示例
for batch in train_loader:
    images, labels = batch
    cwt_images = images['CWT']
    stft_images = images['STFT']
    print(cwt_images.shape)   # [batch_size, 3, 256, 256]
    print(stft_images.shape)  # [batch_size, 3, 256, 256]
    print(labels.shape)       # [batch_size]
    break  # 这里只是示例，运行一次循环后退出

for batch in val_loader:
    images, labels = batch
    print(labels)
    cwt_images = images['CWT']
    stft_images = images['STFT']
    print(cwt_images.shape)   # [batch_size, 3, 256, 256]
    print(stft_images.shape)  # [batch_size, 3, 256, 256]
    print(labels.shape)       # [batch_size]
    break  # 这里只是示例，运行一次循环后退出


torch.Size([64, 3, 227, 227])
torch.Size([64, 3, 227, 227])
torch.Size([64])
tensor([1, 2, 3, 5, 6, 2, 6, 3, 7, 7, 7, 0, 1, 4, 2, 0, 1, 7, 4, 0, 6, 4, 3, 1,
        4, 6, 1, 6, 2, 0, 7, 1, 0, 1, 0, 3, 0, 4, 1, 0, 2, 3, 6, 3, 7, 5, 2, 5,
        2, 0, 2, 6, 0, 0, 0, 2, 6, 3, 0, 4, 8, 3, 5, 5])
torch.Size([64, 3, 227, 227])
torch.Size([64, 3, 227, 227])
torch.Size([64])


In [6]:
from pipeline import *
from models.edgenext import EdgeNeXtDual
import torch.nn as nn
import torch.optim as optim
import torch

criterion = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgeNeXtDual(
                    #  depths=[3, 3, 9, 3], dims=[80, 160, 288, 584], expan_ratio=4,
                    #  global_block=[0, 1, 1, 1],
                    #  global_block_type=['None', 'SDTA', 'SDTA', 'SDTA'],
                    #  use_pos_embd_xca=[False, True, False, False],
                    #  kernel_sizes=[3, 5, 7, 9],
                    #  d2_scales=[2, 2, 3, 4],

                    #  depths=[1, 1, 3, 1], dims=[24, 48, 72, 96], expan_ratio=4,
                    #  global_block=[0, 1, 1, 1],
                    #  global_block_type=['None', 'SDTA', 'SDTA', 'SDTA'],
                    #  use_pos_embd_xca=[False, True, False, False],
                    #  kernel_sizes=[3, 5, 7, 9],
                    #  heads=[4, 4, 4, 4],
                    #  d2_scales=[2, 2, 3, 4],

                    #  depths=[2, 2, 6, 2], dims=[24, 24, 24, 24], expan_ratio=2,
                    #  global_block=[0, 1, 1, 1],
                    #  global_block_type=['None', 'SDTA', 'SDTA', 'SDTA'],
                    #  use_pos_embd_xca=[False, True, False, False],
                    #  kernel_sizes=[3, 3, 3, 3],
                    #  heads=[2, 2, 2, 2],
                    #  d2_scales=[2, 2, 3, 4],

                     depths=[1,1,1,1], dims=[4,4,4,4], expan_ratio=2,
                     global_block=[1, 1, 1, 1],
                     global_block_type=['SDTA', 'SDTA', 'SDTA', 'SDTA'],
                     use_pos_embd_xca=[False, False, False, False],
                     kernel_sizes=[3, 3, 3, 3],
                     heads=[2, 2, 2, 2],
                     d2_scales=[1,1,1,1],
                     
                     num_classes=9,
                     classifier_dropout=0.25,
                     in_chans=3,
                     fusion_method='add',
                     ).to(device)

                     
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

train_and_validate(
        model, 
        train_loader, 
        val_loader, 
        criterion, 
        optimizer, 
        device, 
        num_epochs=200, 
        save_path="best_edgenextdual.pth",
        wandb_log=True,
    )

Epoch [1/200]


IndexError: tuple index out of range